In [15]:
import pandas as pd
import glob
import numpy as np

from scipy import stats

In [16]:
def running_average(arr, window=5):
    # Compute the running average using convolution
    conv_avg = np.convolve(arr, np.ones(window) / window, mode='valid')
    # Prepend and append additional averages as needed
    start_avgs = [np.mean(arr[:3]), np.mean(arr[:4])]
    end_avgs = [np.mean(arr[-4:]), np.mean(arr[-3:])]
    # Combine everything into one array
    return np.concatenate((start_avgs, conv_avg, end_avgs))

In [17]:
def mk_corr_obs(rad, le, sh, gh, swc, month, samplenums=90, bootsnums=1000):
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    var_dict = {    'swc': swc,
                    'rad': rad,
                    'le': le,
                    'sh': sh,
                    'gh': gh}

    # Create month column names as strings '1' to '12'
    columns_mo = [str(i) for i in range(1, 13)]
    results = {}
    
    # Loop over each variable pair in the dictionary.
    for pair_name, (var1_name, var2_name) in pairs.items():
        var1 = var_dict[var1_name]
        var2 = var_dict[var2_name]
        corr_values = []
        # Loop through each month (1 to 12).
        # len(obs)
        for m in range(1, 13):  
            # Filter the data corresponding to the current month.               
            if (len(var1[month == m]) < samplenums):
                corr_mean = np.nan
            else:
                bootstrap = []
                X_m = var1[month == m]
                Y_m = var2[month == m]               

                for _ in range(bootsnums):
                    # Sample indices with replacement
                    idx = np.random.choice(len(X_m), samplenums, replace=True)
                    X_sample = X_m[idx]
                    Y_sample = Y_m[idx]
                    
                    # Compute Pearson correlation for the current month.
                    corr = stats.pearsonr(X_sample, Y_sample)[0]
                    bootstrap.append(corr)
                corr_mean = np.mean(bootstrap)
            corr_values.append(corr_mean)
        # Store the monthly correlations in a DataFrame.
        results[pair_name] = pd.DataFrame([corr_values], columns=columns_mo)
   
    return results

In [18]:
def calc_RR_bins(X):
    """Rice Rule: k = [2 * N ** (1/3)]"""
    num_bins = 2 * (len(X) ** (1/3))    
    return int(num_bins)

def calc_entropy(X):
    """Calculate the Shannon entropy of a dataset."""
    bins = calc_RR_bins(X)
    # print(bins)
    prob = np.histogram(X, bins = int(bins))[0]/len(X)    
    return -np.sum([p * np.log2(p) for p in prob if p > 0])

def calc_joint_entropy(X, Y):
    """Calculate the joint entropy of two datasets."""
    # Determine joint bins using Rice Rule
    bins_x = calc_RR_bins(X)
    bins_y = calc_RR_bins(Y)
    
    # 2D histogram for joint probability
    hist_2d, _, _ = np.histogram2d(X, Y, bins=[bins_x, bins_y])
    prob_2d = hist_2d / np.sum(hist_2d)
    
    # Calculate joint entropy
    return -np.sum([p * np.log2(p) for p in prob_2d.flatten() if p > 0])

def calc_MI(X, Y, samplenums=90, bootsnums=1000):
    """Calculate Mutual Information with bootstrapping."""
    # Filter data for the given month
    X_m = X
    Y_m = Y

    if len(X_m) < samplenums:
        raise ValueError(f"Not enough samples in month {mo} to draw {samplenums} samples.")
    
    mi_bootstrap = []

    for _ in range(bootsnums):
        # Sample indices with replacement
        idx = np.random.choice(len(X_m), samplenums, replace=True)
        X_sample = X_m[idx]
        Y_sample = Y_m[idx]
        
        # Compute MI
        H_x = calc_entropy(X_sample)
        H_y = calc_entropy(Y_sample)
        H_xy = calc_joint_entropy(X_sample, Y_sample)
        I_xy = H_x + H_y - H_xy
        # Normalize MI to [0, 1]
        # NMI = I_xy/min(H_x, H_y)
        NMI = I_xy/(H_x**(0.5) * H_y**(0.5))
        
        mi_bootstrap.append(NMI)
        # mi_bootstrap.append(I_xy)
        
    # Return average MI and optionally confidence intervals
    mi_out = np.mean(mi_bootstrap)
    
    return mi_out

In [19]:
def calc_z(x, y):
    """
    Compute z = Y - Y^
    
    the least-squares regression .

    Parameters:
        x (array-like): Predictor variable (X).
        y (array-like): Response variable (Y).
        ye(array-like): Fitted values (Y^)
    Returns:
        z 
    """
    # Means of X and Y
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    # Calculate slope (m)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean)**2)
    slope = numerator / denominator
    
    # Calculate intercept (b)
    intercept = y_mean - slope * x_mean
    
    # Calculate fitted values Y^
    fitted_values = slope * np.array(x) + intercept
    
    return fitted_values - np.array(y), fitted_values
    
    # return fitted_values

from scipy.stats import norm

def calc_cdf(x):
    x_mean = np.mean(x)
    x_std  = np.std(x)

    return norm.cdf(x , loc=x_mean, scale=x_std)

def calc_quantile_transform(x,y):
    """
    Calculate Y' = F_Y^{-1}(G(z)).
    Smith (2015)
    Parameters:
        x (array-like): Predictor variable (X).
        y (array-like): Response variable (Y).
        z (array-like): Residual values of Y-Y^.
    Returns:
        y_p (array-like): Quantile function (inverse c.d.f.) of G(z)
    """   
    # c.d.f of G(z)
    z     = calc_z (x,y)
    cdf_z = calc_cdf(z[0])
    data = np.sort(y)
    n   = len(data)
    ind = (cdf_z*(n -1)).astype(int)
    # quantile function (inverse c.d.f.) of G(z)
    y_p = data[ind]
    
    return y_p
    

In [20]:
def calc_joint_entropy3(X, Y, Z):
    """
    Calculate the joint entropy of three variables.
    
    Parameters:
        X, Y, Z: Array-like input variables (same length).
    
    Returns:
        float: Joint entropy H(X, Y, Z).
    """
    # Determine bins for each variable
    bins_x = calc_RR_bins(X)
    bins_y = calc_RR_bins(Y)
    bins_z = calc_RR_bins(Z)
    
    # 3D histogram for joint probability
    hist_3d, edges = np.histogramdd(np.vstack((X, Y, Z)).T, bins=[bins_x, bins_y, bins_z])
    prob_3d = hist_3d / np.sum(hist_3d)  # Normalize to get probabilities
    
    # Calculate joint entropy
    joint_entropy = -np.sum([p * np.log2(p) for p in prob_3d.flatten() if p > 0])
    return joint_entropy
    
def calc_transfer_entropy(X, Y, lag=1):
    """
    Calculate the transfer entropy T(X -> Y).
    
    Parameters:
        X (array-like): Source time series.
        Y (array-like): Target time series.
        lag (int): Time lag for past states.
    
    Returns:
        float: Transfer entropy T(X -> Y).
    """
    # Lagged time series
    x_t = X[:-lag]
    y_t = Y[:-lag]
    y_t1 = Y[lag:]  # Future values of Y
    
    # Calculate individual and joint entropies
    H_yt1_yt_xt = calc_joint_entropy3(y_t1, y_t, x_t)
    H_yt_yt_xt = calc_joint_entropy3(y_t, y_t, x_t)
    H_yt1_yt = calc_joint_entropy(y_t1, y_t)
    H_yt = calc_entropy(y_t)
    
    # Transfer Entropy: T(X -> Y)
    TE = -(H_yt1_yt_xt - H_yt_yt_xt - (H_yt1_yt - H_yt))
    return TE

In [21]:
def mk_MI_obs(rad, le, sh, gh, swc, month, samplenums=90):
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    var_dict = {    'swc': swc,
                    'rad': rad,
                    'le': le,
                    'sh': sh,
                    'gh': gh}

    # Create month column names as strings '1' to '12'
    columns_mo = [str(i) for i in range(1, 13)]
    results1 = {}
    results2 = {}
    results3 = {}
    results4 = {}
    results5 = {}
    
    # Loop over each variable pair in the dictionary.
    for pair_name, (var1_name, var2_name) in pairs.items():
        var1 = var_dict[var1_name]
        var2 = var_dict[var2_name]
        nmi_values = []
        nmi_nonlinear_values = []
        nmi_linear_values = []
        te_x_y_values = []
        te_y_x_values = []
        
        # Loop through each month (1 to 12).
        # len(obs)
        for m in range(1, 13):  
            # Filter the data corresponding to the current month.               
            if (len(var1[month == m]) < samplenums):
                nmi = np.nan
                nmi_nonlinear = np.nan
                nmi_linear = np.nan
                te_x_y = np.nan
                te_y_x = np.nan
            else:
                # Compute Pearson correlation for the current month.
                X  = var1[month==m]
                Y  = var2[month==m]
                # print(X)
                nmi = calc_MI(X,Y)
                Y_p= calc_quantile_transform(X,Y)
                nmi_nonlinear = calc_MI(X,Y_p)
                nmi_linear    = nmi - nmi_nonlinear
                te_x_y        = calc_transfer_entropy(X,Y)
                te_y_x        = calc_transfer_entropy(Y,X)
                
            nmi_values.append(nmi)
            nmi_nonlinear_values.append(nmi_nonlinear)
            nmi_linear_values.append(nmi_linear)
            te_x_y_values.append(te_x_y)
            te_y_x_values.append(te_y_x)
            
        # Store the monthly correlations in a DataFrame.
        results1[pair_name] = pd.DataFrame([nmi_values], columns=columns_mo)
        results2[pair_name] = pd.DataFrame([nmi_nonlinear_values], columns=columns_mo)
        results3[pair_name] = pd.DataFrame([nmi_linear_values], columns=columns_mo)
        results4[pair_name] = pd.DataFrame([te_x_y_values], columns=columns_mo)
        results5[pair_name] = pd.DataFrame([te_y_x_values], columns=columns_mo)
           
    return results1,results2,results3,results4,results5

In [22]:
site_obs = ['US-A03', 'US-A10', 'US-A32', 'US-A74', 'US-ADR', 'US-AR1', 'US-AR2', 'US-ARM', 'US-BMM', 'US-Bi1', 'US-Bi2', 'US-Bo1', 'US-Bo2', 'US-Bsg', 'US-CGG', 'US-CPk', 'US-CRT', 'US-CS2', 'US-Ced', 'US-Cst', 'US-DFC', 'US-DFK', 'US-Dia', 'US-Dix', 'US-Elm', 'US-Esm', 'US-Fcr', 'US-Fmf', 'US-Fuf', 'US-Fwf', 'US-GBT', 'US-GLE', 'US-HB2', 'US-HB3', 'US-HBK', 'US-HRA', 'US-HRC', 'US-HWB', 'US-Hn2', 'US-Hn3', 'US-Ho1', 'US-Ho3', 'US-IB1', 'US-IB2', 'US-ICh', 'US-ICs', 'US-ICt', 'US-Jo1', 'US-Jo2', 'US-KFS', 'US-KLS', 'US-KM4', 'US-KUT', 'US-Kon', 'US-Lin', 'US-MC1', 'US-MC2', 'US-MH1', 'US-MH2', 'US-MN3', 'US-MOz', 'US-MSR', 'US-MVW', 'US-Me1', 'US-Me2', 'US-Me5', 'US-Me6', 'US-Mi1', 'US-Mi2', 'US-Mi3', 'US-Mj1', 'US-Mj2', 'US-Mo1', 'US-Mo2', 'US-Mo3', 'US-MtB', 'US-NC1', 'US-NC2', 'US-NC3', 'US-NC4', 'US-NGC', 'US-NR3', 'US-NR4', 'US-ONA', 'US-Oho', 'US-PFL', 'US-PFb', 'US-PFc', 'US-PFd', 'US-PFe', 'US-PFf', 'US-PFg', 'US-PFh', 'US-PFi', 'US-PFj', 'US-PFk', 'US-PFm', 'US-PFn', 'US-PFp', 'US-PFq', 'US-PFr', 'US-PFs', 'US-PFt', 'US-Rls', 'US-Rms', 'US-Ro1', 'US-Ro4', 'US-Ro5', 'US-Ro6', 'US-Rpf', 'US-Rws', 'US-SP1', 'US-SP2', 'US-SP3', 'US-SRC', 'US-SRG', 'US-SRS', 'US-SdH', 'US-Slt', 'US-Snd', 'US-Syv', 'US-Ton', 'US-Tur', 'US-Tw2', 'US-Tw3', 'US-UC1', 'US-UC2', 'US-UiA', 'US-UiB', 'US-UiC', 'US-UiD', 'US-Var', 'US-WCr', 'US-Whs', 'US-Wkg', 'US-Wlr', 'US-Wrc', 'US-YK1', 'US-YK2', 'US-xAB', 'US-xAE', 'US-xBA', 'US-xBL', 'US-xBN', 'US-xBR', 'US-xCL', 'US-xCP', 'US-xDC', 'US-xDJ', 'US-xDL', 'US-xDS', 'US-xGR', 'US-xHA', 'US-xHE', 'US-xJE', 'US-xJR', 'US-xKA', 'US-xKZ', 'US-xLE', 'US-xMB', 'US-xML', 'US-xNG', 'US-xNQ', 'US-xNW', 'US-xRM', 'US-xRN', 'US-xSB', 'US-xSC', 'US-xSE', 'US-xSJ', 'US-xSL', 'US-xSP', 'US-xSR', 'US-xST', 'US-xTA', 'US-xTE', 'US-xTL', 'US-xTR', 'US-xUK', 'US-xUN', 'US-xWD', 'US-xWR', 'US-xYE']

In [23]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def mk_obs_mo_r(time_scale):
    dir = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/'
    site_list = site_obs
    f_start = 0 ; f_stop = len(site_list)
    
    columns = ['NETRAD_1_A','LE_1_A','H_1_A','G_1_A','SWC_1_A']
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    
    columns_mo   = [str(i) for i in range(1, 13)]
    results_out  = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out2 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out3 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out4 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out5 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out6 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
     
    for s,site in enumerate(site_list[0:f_stop]):
    # for s,site in enumerate(site_list[0:5]):
        f_siteid = site_list[s]
        # print(f_siteid)
        df   = pd.read_csv(dir+f_siteid+'.csv',na_values=-9999)
        dir_loc = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/site/'
        df_loc  = pd.read_csv(dir_loc+f_siteid+'_site.csv',na_values=-9999)
        
        lat     = df_loc['Latitude'].item()
        if float(lat) < 50:                
            tm      = df['TIME']    
            yrs     = str(tm[0])[:4]   ;    yre     = str(tm[int(len(tm)-1)])[:4]
            mos     = str(tm[0])[4:6]  ;    moe     = str(tm[int(len(tm)-1)])[4:6]
            dys     = str(tm[0])[6:8]  ;    dye     = str(tm[int(len(tm)-1)])[6:8]
            hrs     = str(tm[0])[8:10] ;    hre     = str(tm[int(len(tm)-1)])[8:10]
            mns     = str(tm[0])[10:12];    mne     = str(tm[int(len(tm)-1)])[10:12]

            time = pd.date_range(yrs+'-'+mos+'-'+dys+' '+hrs+':'+mns,yre+'-'+moe+'-'+dye+' '+hre+':'+mne,freq='30min')        
            df.index = pd.to_datetime(time)
            
            rad    = running_average(df[columns[0]].resample(time_scale).mean())   
            le     = running_average(df[columns[1]].resample(time_scale).mean())
            sh     = running_average(df[columns[2]].resample(time_scale).mean())
            gh     = running_average(df[columns[3]].resample(time_scale).mean())
            swc    = running_average(df[columns[4]].resample(time_scale).mean())

            time_dy = pd.date_range(yrs+'-'+mos+'-'+dys+' '+hrs+':'+mns,yre+'-'+moe+'-'+dye+' '+hre+':'+mne,freq='1d')        

            rad = pd.Series(rad, index=pd.to_datetime(time_dy))
            le  = pd.Series(le, index=pd.to_datetime(time_dy))
            sh  = pd.Series(sh, index=pd.to_datetime(time_dy))
            gh  = pd.Series(gh, index=pd.to_datetime(time_dy))
            swc = pd.Series(swc, index=pd.to_datetime(time_dy))
            # # no running avg option
            # rad    = df[columns[0]].resample(time_scale).mean()  
            # le     = df[columns[1]].resample(time_scale).mean()  
            # sh     = df[columns[2]].resample(time_scale).mean()  
            # gh     = df[columns[3]].resample(time_scale).mean()  
            # swc    = df[columns[4]].resample(time_scale).mean()  
            
            
            sort_nan = ~np.logical_or.reduce((np.isnan(rad),np.isnan(le),np.isnan(sh),np.isnan(gh),np.isnan(swc)))

            month = rad.index.month[sort_nan]
            rad   = rad[sort_nan]
            le    = le [sort_nan]
            sh    = sh [sort_nan]
            gh    = gh [sort_nan]
            swc   = swc[sort_nan]

            if int(yre) > 2020:
                month = month[rad.index.year <= 2020]
                rad   = rad[rad.index.year <= 2020]
                le    = le [le.index.year <= 2020]
                sh    = sh [sh.index.year <= 2020]
                gh    = gh [gh.index.year <= 2020]
                swc   = swc[swc.index.year <= 2020]

            if int(yrs) < 2021:
                # corr = mk_corr_obs(rad, le, sh, gh, swc, month)
                mi   = mk_MI_obs(rad, le, sh, gh, swc, month)
                # print(mi[0])
            for pair_name in pairs:
                # results_out [pair_name] = pd.concat([results_out [pair_name], corr[pair_name]],ignore_index=True)
                results_out2[pair_name] = pd.concat([results_out2[pair_name], mi[0][pair_name]],ignore_index=True)
                results_out3[pair_name] = pd.concat([results_out3[pair_name], mi[1][pair_name]],ignore_index=True)
                results_out4[pair_name] = pd.concat([results_out4[pair_name], mi[2][pair_name]],ignore_index=True)
                results_out5[pair_name] = pd.concat([results_out5[pair_name], mi[3][pair_name]],ignore_index=True)
                results_out6[pair_name] = pd.concat([results_out6[pair_name], mi[4][pair_name]],ignore_index=True)
                

    print('done')
    return  results_out2,results_out3,results_out4,results_out5,results_out6
    # return  results_out, results_out2

In [24]:
obs = mk_obs_mo_r('1D')

done


In [25]:
def mk_sort_nan(site):
    dir     = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/'
    # dir     = '/projects/COLA/land/skim/ameribase_skim/'
    columns = ['NETRAD_1_A','LE_1_A','H_1_A','G_1_A','SWC_1_A']
    df   = pd.read_csv(dir+site+'.csv',na_values=-9999)    
        
    tm      = df['TIME']    
    yrs     = str(tm[0])[:4]   ;    yre     = str(tm[int(len(tm)-1)])[:4]
    mos     = str(tm[0])[4:6]  ;    moe     = str(tm[int(len(tm)-1)])[4:6]
    dys     = str(tm[0])[6:8]  ;    dye     = str(tm[int(len(tm)-1)])[6:8]
    hrs     = str(tm[0])[8:10] ;    hre     = str(tm[int(len(tm)-1)])[8:10]
    mns     = str(tm[0])[10:12];    mne     = str(tm[int(len(tm)-1)])[10:12]

    time = pd.date_range(yrs+'-'+mos+'-'+dys+' '+hrs+':'+mns,yre+'-'+moe+'-'+dye+' '+hre+':'+mne,freq='30MIN')        
    df.index = pd.to_datetime(time)

    rad    = df[columns[0]].resample('1D').mean()    
    le     = df[columns[1]].resample('1D').mean()
    sh     = df[columns[2]].resample('1D').mean()
    gh     = df[columns[3]].resample('1D').mean()
    swc1   = df[columns[4]].resample('1D').mean()

    if int(yre) > 2020:            
        rad = rad[rad.index.year <= 2020]
        le  = le [le.index.year <= 2020]
        sh  = sh [sh.index.year <= 2020]
        gh  = gh [gh.index.year <= 2020]
        swc1= swc1[swc1.index.year <= 2020]

        
    sort_nan = ~np.logical_or.reduce((np.isnan(rad),np.isnan(le),np.isnan(sh),np.isnan(gh),np.isnan(swc1)))
    
    return sort_nan

In [26]:
def mk_era_mo_r(time_scale):
    site_list = site_obs
    f_start = 0 ; f_stop = len(site_list)
    
    columns = ['NETRAD_1_A','LE_1_A','H_1_A','G_1_A','SWC_1_A']
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    
    columns_mo = [str(i) for i in range(1, 13)]
    results_out = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out2 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out3 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out4 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out5 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out6 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
     
    for s,site in enumerate(site_list[:]):
    # for s,site in enumerate(site_list[8:9]):
        f_siteid = site_list[s]
        
        dir_loc = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/site/'
        df_loc  = pd.read_csv(dir_loc+f_siteid+'_site.csv',na_values=-9999)
        
        lat     = df_loc['Latitude'].item()
        if float(lat) < 50:
            dir1  = '/projects/COLA/land/skim/FLUXNET/AMERIbase/era5/out_v2'
            df1   = pd.read_csv(dir1+'/LE_1_A/'+f_siteid+'.csv',na_values=-9999)
            df2   = pd.read_csv(dir1+'/H_1_A/'+f_siteid+'.csv',na_values=-9999)
            # df3   = pd.read_csv(dir1+'/GH_1_A/'+f_siteid+'.csv',na_values=-9999)
            df4   = pd.read_csv(dir1+'/SWC_1_A/'+f_siteid+'.csv',na_values=-9999)
            df5   = pd.read_csv(dir1+'/SW_1_A/'+f_siteid+'.csv',na_values=-9999)
            df6   = pd.read_csv(dir1+'/LW_1_A/'+f_siteid+'.csv',na_values=-9999)

            tm      = df1['TIME']    
            yrs     = str(tm[0])[:4]   ;    yre     = str(tm[int(len(tm)-1)])[:4]
            mos     = str(tm[0])[5:7]  ;    moe     = str(tm[int(len(tm)-1)])[5:7]
            dys     = str(tm[0])[8:10]  ;    dye     = str(tm[int(len(tm)-1)])[8:10]
            hrs     = str(tm[0])[11:13] ;    hre     = str(tm[int(len(tm)-1)])[11:13]
            mns     = str(tm[0])[14:16];    mne     = str(tm[int(len(tm)-1)])[14:16]
            # print(f_siteid,yrs, mos, yre, moe)

            time  = pd.date_range(yrs+'-'+mos+'-'+dys,yre+'-'+moe+'-'+dye,freq='1D')   
            df1.index = pd.to_datetime(time)
            df2.index = pd.to_datetime(time)
            # df3.index = pd.to_datetime(time)
            df4.index = pd.to_datetime(time)
            df5.index = pd.to_datetime(time)
            df6.index = pd.to_datetime(time)  

            le   = df1['LE_1_A']
            sh   = df2['H_1_A']
            # gh   = df3['GH_1_A'][sort_nan]
            swc  = df4['SWC_1_A']
            sw   = df5['SW_1_A']
            lw   = df6['LW_1_A']
            rad  = sw + lw
            gh   = rad - (le + sh)

            rad    = running_average(rad)   
            le     = running_average(le)
            sh     = running_average(sh)
            gh     = running_average(gh)
            swc    = running_average(swc)

            rad = pd.Series(rad, index=pd.to_datetime(time))
            le  = pd.Series(le, index=pd.to_datetime(time))
            sh  = pd.Series(sh, index=pd.to_datetime(time))
            gh  = pd.Series(gh, index=pd.to_datetime(time))
            swc = pd.Series(swc, index=pd.to_datetime(time))

            month = le.index.month
            sort_nan = mk_sort_nan(f_siteid)

            if (len(le.values) > len(sort_nan)):
                min_len = min(len(month.values), len(sort_nan))
                month = month[:min_len]
                rad   = rad[:min_len]
                le    = le [:min_len]
                sh    = sh [:min_len]
                gh    = gh [:min_len]
                swc   = swc[:min_len]                

                month = month[sort_nan]
                rad   = rad[sort_nan]
                le    = le [sort_nan]
                sh    = sh [sort_nan]
                gh    = gh [sort_nan]
                swc   = swc[sort_nan]

                del min_len
            else:
                month = month[sort_nan]
                rad   = rad[sort_nan]
                le    = le [sort_nan]
                sh    = sh [sort_nan]
                gh    = gh [sort_nan]
                swc   = swc[sort_nan]

            # corr = mk_corr_obs(rad, le, sh, gh, swc, month)
            mi   = mk_MI_obs(rad, le, sh, gh, swc, month)
                
            for pair_name in pairs:
                # results_out[pair_name] = pd.concat([results_out[pair_name], corr[pair_name]],ignore_index=True)
                results_out2[pair_name] = pd.concat([results_out2[pair_name], mi[0][pair_name]],ignore_index=True)
                results_out3[pair_name] = pd.concat([results_out3[pair_name], mi[1][pair_name]],ignore_index=True)
                results_out4[pair_name] = pd.concat([results_out4[pair_name], mi[2][pair_name]],ignore_index=True)
                results_out5[pair_name] = pd.concat([results_out5[pair_name], mi[3][pair_name]],ignore_index=True)
                results_out6[pair_name] = pd.concat([results_out6[pair_name], mi[4][pair_name]],ignore_index=True)
                

    print('done')
    return  results_out2,results_out3,results_out4,results_out5,results_out6
    # return  results_out, results_out2 


In [27]:
era = mk_era_mo_r('1D')

done


In [28]:
def mk_merra_mo_r(time_scale):
    site_list = site_obs
    f_start = 0 ; f_stop = len(site_list)
    
    columns = ['NETRAD_1_A','LE_1_A','H_1_A','G_1_A','SWC_1_A']
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    
    columns_mo = [str(i) for i in range(1, 13)]
    results_out = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out2 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out3 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out4 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out5 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out6 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
       
    for s,site in enumerate(site_list[:]):
    # for s,site in enumerate(site_list[8:9]):
        f_siteid = site_list[s]
        
        dir_loc = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/site/'
        df_loc  = pd.read_csv(dir_loc+f_siteid+'_site.csv',na_values=-9999)
        
        lat     = df_loc['Latitude'].item()
        if float(lat) < 50:
            dir1  = '/projects/COLA/land/skim/FLUXNET/AMERIbase/merra2/out_v2'
            df1   = pd.read_csv(dir1+'/LE_1_A/'+f_siteid+'.csv',na_values=-9999)
            df2   = pd.read_csv(dir1+'/H_1_A/'+f_siteid+'.csv',na_values=-9999)
            df3   = pd.read_csv(dir1+'/GH_1_A/'+f_siteid+'.csv',na_values=-9999)
            df4   = pd.read_csv(dir1+'/SWC_1_A/'+f_siteid+'.csv',na_values=-9999)
            df5   = pd.read_csv(dir1+'/SW_1_A/'+f_siteid+'.csv',na_values=-9999)
            df6   = pd.read_csv(dir1+'/LW_1_A/'+f_siteid+'.csv',na_values=-9999)

            tm      = df1['TIME']    
            yrs     = str(tm[0])[:4]   ;    yre     = str(tm[int(len(tm)-1)])[:4]
            mos     = str(tm[0])[5:7]  ;    moe     = str(tm[int(len(tm)-1)])[5:7]
            dys     = str(tm[0])[8:10]  ;    dye     = str(tm[int(len(tm)-1)])[8:10]
            hrs     = str(tm[0])[11:13] ;    hre     = str(tm[int(len(tm)-1)])[11:13]
            mns     = str(tm[0])[14:16];    mne     = str(tm[int(len(tm)-1)])[14:16]
            # print(f_siteid,yrs, mos, yre, moe)

            time  = pd.date_range(yrs+'-'+mos+'-'+dys,yre+'-'+moe+'-'+dye,freq='1D')   
            df1.index = pd.to_datetime(time)
            df2.index = pd.to_datetime(time)
            df3.index = pd.to_datetime(time)
            df4.index = pd.to_datetime(time)
            df5.index = pd.to_datetime(time)
            df6.index = pd.to_datetime(time)  

            if (df1.columns[1] == 'GH_1_A'):
                le   = df1['GH_1_A']
            else:
                le   = df1['LE_1_A']

            if (df2.columns[1] == 'GH_1_A'):
                sh   = df2['GH_1_A']
            else:
                sh   = df2['H_1_A']

            if (df4.columns[1] == 'GH_1_A'):
                swc   = df4['GH_1_A']
            else:
                swc   = df4['SWC_1_A']

            if (df5.columns[1] == 'GH_1_A'):
                sw   = df5['GH_1_A']
            else:
                sw   = df5['SW_1_A']

            if (df6.columns[1] == 'GH_1_A'):
                lw   = df6['GH_1_A']
            else:
                lw   = df6['LW_1_A']

            gh   = df3['GH_1_A']
            rad  = sw + lw
            # gh   = rad - (le + sh)
            rad    = running_average(rad)   
            le     = running_average(le)
            sh     = running_average(sh)
            gh     = running_average(gh)
            swc    = running_average(swc)

            rad = pd.Series(rad, index=pd.to_datetime(time))
            le  = pd.Series(le, index=pd.to_datetime(time))
            sh  = pd.Series(sh, index=pd.to_datetime(time))
            gh  = pd.Series(gh, index=pd.to_datetime(time))
            swc = pd.Series(swc, index=pd.to_datetime(time))

            month = le.index.month
            sort_nan = mk_sort_nan(f_siteid)


            if (len(le.values) > len(sort_nan)):
                min_len = min(len(month.values), len(sort_nan))
                month = month[:min_len]
                rad   = rad[:min_len]
                le    = le [:min_len]
                sh    = sh [:min_len]
                gh    = gh [:min_len]
                swc   = swc[:min_len]                

                month = month[sort_nan]
                rad   = rad[sort_nan]
                le    = le [sort_nan]
                sh    = sh [sort_nan]
                gh    = gh [sort_nan]
                swc   = swc[sort_nan]

                del min_len
            else:
                month = month[sort_nan]
                rad   = rad[sort_nan]
                le    = le [sort_nan]
                sh    = sh [sort_nan]
                gh    = gh [sort_nan]
                swc   = swc[sort_nan]
    
            # corr = mk_corr_obs(rad, le, sh, gh, swc, month)
            mi   = mk_MI_obs(rad, le, sh, gh, swc, month)
                
            for pair_name in pairs:
                # results_out[pair_name] = pd.concat([results_out[pair_name], corr[pair_name]],ignore_index=True)
                results_out2[pair_name] = pd.concat([results_out2[pair_name], mi[0][pair_name]],ignore_index=True)
                results_out3[pair_name] = pd.concat([results_out3[pair_name], mi[1][pair_name]],ignore_index=True)
                results_out4[pair_name] = pd.concat([results_out4[pair_name], mi[2][pair_name]],ignore_index=True)
                results_out5[pair_name] = pd.concat([results_out5[pair_name], mi[3][pair_name]],ignore_index=True)
                results_out6[pair_name] = pd.concat([results_out6[pair_name], mi[4][pair_name]],ignore_index=True)
                

    print('done')
    return  results_out2,results_out3,results_out4,results_out5,results_out6
    # return  results_out, results_out2   
    # print('done')
    # return results_out,results_out2

In [29]:
merra = mk_merra_mo_r('1D')

done


In [30]:
import math
# def corr_mo(var1, var2, month, mo):
#     # 99% confidence lv. p 0.01
#     # 95% confidence lv. p 0.03
#     # 95% confidence lv. p 0.1
    
#     # if (stats.pearsonr(var1[month == mo],var2[month == mo])[1] <= 0.01):
#     corr= stats.pearsonr(var1[month == mo],var2[month == mo])[0]
#     # else:            
#         # corr= np.nan
        
#     return corr

def mk_corr_cesm(s, rad, le, sh, gh, swc, month, samplenums=90, bootsnums=1000):
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    var_dict = {    'swc': swc,
                    'rad': rad,
                    'le': le,
                    'sh': sh,
                    'gh': gh}

    # Create month column names as strings '1' to '12'
    columns_mo = [str(i) for i in range(1, 13)]
    results = {}
    dir_obs = '/projects/COLA/land/skim/FLUXNET/AMERIbase/'
    obs     = pd.read_csv(dir_obs+'obs_chck.csv',na_values=-9999)
    
    # Loop over each variable pair in the dictionary.
    for pair_name, (var1_name, var2_name) in pairs.items():
        df1 = var_dict[var1_name].reset_index()
        df2 = var_dict[var2_name].reset_index()

        corr_values = []
        month_values = []

        df1['TIME']  = df1['TIME'].astype(str)
        df1['MONTH'] = df1['TIME'].str.slice(5, 7).astype(int)   # Extract the month
        df2['TIME']  = df2['TIME'].astype(str)
        df2['MONTH'] = df2['TIME'].str.slice(5, 7).astype(int)   # Extract the month

        vars1 = df1.groupby('MONTH')[0]
        vars2 = df2.groupby('MONTH')[0]
      
        # Loop through each month (1 to 12).
        for m in range(1, 13):
            value = obs.iloc[s][str(m)]
            if value is None or math.isnan(value):  # Skip if the value is None or NaN
                 corr_mean = np.nan
            else:
                bootstrap = []
                # print(vars1)
                X  = vars1.get_group(m).values
                Y  = vars2.get_group(m).values
                sort_nan = ~np.logical_or.reduce((np.isnan(X),np.isnan(Y)))
                X_m  = X[sort_nan]
                Y_m  = Y[sort_nan]
                # X_m = vars1[month == m]
                # Y_m = vars2[month == m] 
                
                if len(X_m > 1):
                    for _ in range(bootsnums):
                        # Sample indices with replacement
                        idx = np.random.choice(len(X_m), samplenums, replace=True)
                        X_sample = X_m[idx]
                        Y_sample = Y_m[idx]

                        # Compute Pearson correlation for the current month.
                        corr = stats.pearsonr(X_sample, Y_sample)[0]
                        bootstrap.append(corr)
                    corr_mean = np.mean(bootstrap)
                else:
                    corr_mean = np.nan
                    
            corr_values.append(corr_mean)
        # Store the monthly correlations in a DataFrame.
        results[pair_name] = pd.DataFrame([corr_values], columns=columns_mo)

   
    return results

In [31]:
def mk_MI_cesm(s, rad, le, sh, gh, swc, month, samplenums=90):
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    var_dict = {    'swc': swc,
                    'rad': rad,
                    'le': le,
                    'sh': sh,
                    'gh': gh}

    # Create month column names as strings '1' to '12'
    columns_mo = [str(i) for i in range(1, 13)]
    results1 = {}
    results2 = {}
    results3 = {}
    results4 = {}
    results5 = {}
    
    
    dir_obs = '/projects/COLA/land/skim/FLUXNET/AMERIbase/'
    obs     = pd.read_csv(dir_obs+'obs_chck.csv',na_values=-9999)
    
    # Loop over each variable pair in the dictionary.
    for pair_name, (var1_name, var2_name) in pairs.items():
        df1 = var_dict[var1_name].reset_index()
        df2 = var_dict[var2_name].reset_index()

        nmi_values = []
        nmi_nonlinear_values = []
        nmi_linear_values = []
        te_x_y_values = []
        te_y_x_values = []
        month_values = []

        df1['TIME']  = df1['TIME'].astype(str)
        df1['MONTH'] = df1['TIME'].str.slice(5, 7).astype(int)   # Extract the month
        df2['TIME']  = df2['TIME'].astype(str)
        df2['MONTH'] = df2['TIME'].str.slice(5, 7).astype(int)   # Extract the month

        vars1 = df1.groupby('MONTH')[0]
        vars2 = df2.groupby('MONTH')[0]
      
        # Loop through each month (1 to 12).
        for m in range(1, 13):
            value = obs.iloc[s][str(m)]
            if value is None or math.isnan(value):  # Skip if the value is None or NaN
                nmi = np.nan
                nmi_nonlinear =  np.nan
                nmi_linear =  np.nan
                te_x_y =  np.nan
                te_y_x =  np.nan
            else:
                # Compute Pearson correlation for the current month.
                X  = vars1.get_group(m).values
                Y  = vars2.get_group(m).values
                sort_nan = ~np.logical_or.reduce((np.isnan(X),np.isnan(Y)))
                X  = X[sort_nan]
                Y  = Y[sort_nan]
                if len(X) > samplenums:
                    nmi = calc_MI(X,Y)
                    Y_p= calc_quantile_transform(X,Y)
                    nmi_nonlinear = calc_MI(X,Y_p)
                    nmi_linear    = nmi - nmi_nonlinear
                    te_x_y        = calc_transfer_entropy(X,Y)
                    te_y_x        = calc_transfer_entropy(Y,X)
                else:
                    nmi = np.nan
                    nmi_nonlinear =  np.nan
                    nmi_linear =  np.nan
                    te_x_y =  np.nan
                    te_y_x =  np.nan
                
            nmi_values.append(nmi)
            nmi_nonlinear_values.append(nmi_nonlinear)
            nmi_linear_values.append(nmi_linear)
            te_x_y_values.append(te_x_y)
            te_y_x_values.append(te_y_x)
            
        # Store the monthly correlations in a DataFrame.
        results1[pair_name] = pd.DataFrame([nmi_values], columns=columns_mo)
        results2[pair_name] = pd.DataFrame([nmi_nonlinear_values], columns=columns_mo)
        results3[pair_name] = pd.DataFrame([nmi_linear_values], columns=columns_mo)
        results4[pair_name] = pd.DataFrame([te_x_y_values], columns=columns_mo)
        results5[pair_name] = pd.DataFrame([te_y_x_values], columns=columns_mo)
       

    # print('done')
   
    return results1,results2,results3,results4,results5

In [32]:
from datetime import datetime
import math 
def mk_clm_mo_r(time_scale):
    site_list = site_obs
    f_start = 0 ; f_stop = len(site_list)
    
    columns = ['NETRAD_1_A','LE_1_A','H_1_A','G_1_A','SWC_1_A']
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    
    columns_mo = [str(i) for i in range(1, 13)]
    results_out = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out2 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out3 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out4 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out5 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out6 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    for s,site in enumerate(site_list[:]):
    # for s,site in enumerate(site_list[1:5]):
        f_siteid = site_list[s]
        
        dir_loc = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/site/'
        df_loc  = pd.read_csv(dir_loc+f_siteid+'_site.csv',na_values=-9999)
        
        lat     = df_loc['Latitude'].item()
        if float(lat) < 50:
            # print(f_siteid)
            dir1= '/projects/COLA/land/skim/FLUXNET/AMERIbase/cesm/clm/out_v2'
            model_len = 8761
            df1   = pd.read_csv(dir1+'/LE_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df2   = pd.read_csv(dir1+'/H_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df3   = pd.read_csv(dir1+'/GH_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df4   = pd.read_csv(dir1+'/SWC_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df5   = pd.read_csv(dir1+'/Rn_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]

            df1['TIME'] = df1['TIME'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
            df1.index = pd.PeriodIndex([t.strftime("%Y-%m-%d") for t in df1['TIME']], freq='D')
            month = df1.index.month

            df1.set_index("TIME", inplace=True)
            df2.set_index("TIME", inplace=True)
            df3.set_index("TIME", inplace=True)
            df4.set_index("TIME", inplace=True)
            df5.set_index("TIME", inplace=True)
            
            # rad = df5['Rn_1_A']
            # le  = df1['LE_1_A']
            # sh  = df2['H_1_A']
            # gh  = df3['GH_1_A']
            # swc = df4['SWC_1_A']

            le     = running_average(df1['LE_1_A'])
            sh     = running_average(df2['H_1_A'])
            gh     = running_average(df3['GH_1_A'])
            swc    = running_average(df4['SWC_1_A'])
            rad    = running_average(df5['Rn_1_A'])  
            # print(le)
            
            le  = pd.Series(le , index=df1.index)
            sh  = pd.Series(sh , index=df2.index)
            gh  = pd.Series(gh , index=df3.index)
            swc = pd.Series(swc, index=df4.index)
            rad = pd.Series(rad, index=df5.index)
   
            # corr = mk_corr_cesm(s, rad, le, sh, gh, swc, month)
            mi   = mk_MI_cesm(s, rad, le, sh, gh, swc, month)
                
            for pair_name in pairs:
                # results_out[pair_name] = pd.concat([results_out[pair_name], corr[pair_name]],ignore_index=True)
                results_out2[pair_name] = pd.concat([results_out2[pair_name], mi[0][pair_name]],ignore_index=True)
                results_out3[pair_name] = pd.concat([results_out3[pair_name], mi[1][pair_name]],ignore_index=True)
                results_out4[pair_name] = pd.concat([results_out4[pair_name], mi[2][pair_name]],ignore_index=True)
                results_out5[pair_name] = pd.concat([results_out5[pair_name], mi[3][pair_name]],ignore_index=True)
                results_out6[pair_name] = pd.concat([results_out6[pair_name], mi[4][pair_name]],ignore_index=True)
                

    # print('done')
    return  results_out2,results_out3,results_out4,results_out5,results_out6    
    # print('done')
    # return results_out, results_out2

In [ ]:
clm = mk_clm_mo_r('1D')

In [ ]:
def mk_amip_mo_r(time_scale):
    site_list = site_obs
    f_start = 0 ; f_stop = len(site_list)
    
    columns = ['NETRAD_1_A','LE_1_A','H_1_A','G_1_A','SWC_1_A']
    pairs = {   'swc_rad': ('swc', 'rad'),
                'swc_le':  ('swc', 'le'),
                'swc_sh':  ('swc', 'sh'),
                'swc_gh':  ('swc', 'gh'),
                'rad_le':  ('rad', 'le'),
                'rad_sh':  ('rad', 'sh'),
                'rad_gh':  ('rad', 'gh'),
                'le_sh':   ('le', 'sh'),
                'le_gh':   ('le', 'gh'),
                'sh_gh':   ('sh', 'gh'),}
    
    columns_mo = [str(i) for i in range(1, 13)]
    results_out = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out2 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out3 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out4 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out5 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    results_out6 = {pair_name: pd.DataFrame(columns=columns_mo) for pair_name in pairs.keys()}
    
    for s,site in enumerate(site_list[:]):
    # for s,site in enumerate(site_list[1:5]):
        f_siteid = site_list[s]
        
        dir_loc = '/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/site/'
        df_loc  = pd.read_csv(dir_loc+f_siteid+'_site.csv',na_values=-9999)
        
        lat     = df_loc['Latitude'].item()
        if float(lat) < 50:
            # print(f_siteid)
            dir1= '/projects/COLA/land/skim/FLUXNET/AMERIbase/cesm/amip/out_v2'
            model_len = 8761
            df1   = pd.read_csv(dir1+'/LE_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df2   = pd.read_csv(dir1+'/H_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df3   = pd.read_csv(dir1+'/GH_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df4   = pd.read_csv(dir1+'/SWC_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]
            df5   = pd.read_csv(dir1+'/Rn_1_A/'+f_siteid+'.csv',na_values=-9999)[:model_len]

            df1['TIME'] = df1['TIME'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
            df1.index = pd.PeriodIndex([t.strftime("%Y-%m-%d") for t in df1['TIME']], freq='D')
            month = df1.index.month
            
            df1.set_index("TIME", inplace=True)
            df2.set_index("TIME", inplace=True)
            df3.set_index("TIME", inplace=True)
            df4.set_index("TIME", inplace=True)
            df5.set_index("TIME", inplace=True)
            

            le     = running_average(df1['LE_1_A'])
            sh     = running_average(df2['H_1_A'])
            gh     = running_average(df3['GH_1_A'])
            swc    = running_average(df4['SWC_1_A'])
            rad    = running_average(df5['Rn_1_A'])   

            
            le  = pd.Series(le , index=df1.index)
            sh  = pd.Series(sh , index=df2.index)
            gh  = pd.Series(gh , index=df3.index)
            swc = pd.Series(swc, index=df4.index)
            rad = pd.Series(rad, index=df5.index)
            # print(rad,le,sh,gh,swc)

            # corr = mk_corr_cesm(s, rad, le, sh, gh, swc, month)
            mi   = mk_MI_cesm(s, rad, le, sh, gh, swc, month)
                
            for pair_name in pairs:
                # results_out[pair_name] = pd.concat([results_out[pair_name], corr[pair_name]],ignore_index=True)
                results_out2[pair_name] = pd.concat([results_out2[pair_name], mi[0][pair_name]],ignore_index=True)
                results_out3[pair_name] = pd.concat([results_out3[pair_name], mi[1][pair_name]],ignore_index=True)
                results_out4[pair_name] = pd.concat([results_out4[pair_name], mi[2][pair_name]],ignore_index=True)
                results_out5[pair_name] = pd.concat([results_out5[pair_name], mi[3][pair_name]],ignore_index=True)
                results_out6[pair_name] = pd.concat([results_out6[pair_name], mi[4][pair_name]],ignore_index=True)
                

    # print('done')
    return  results_out2,results_out3,results_out4,results_out5,results_out6    
    # print('done')
    # return results_out, results_out2

In [ ]:
amip= mk_amip_mo_r('1D')

In [ ]:
def compute_summary(corr_dict):
    summary_dict = {}
    for pair_name, df in corr_dict.items():
        # Compute quartiles and median for each column and create a summary DataFrame.
        summary_df = pd.DataFrame({'Q1': df.quantile(0.25),
                                    'median': df.median(),
                                    'Q3': df.quantile(0.75)}).T  # Transpose so that the index is ['Q1', 'median', 'Q3'].
        summary_dict[pair_name] = summary_df
    return summary_dict

In [ ]:
# Compute summaries for each correlation dictionary.
obs_corr_summary   = compute_summary(obs[0])
era_corr_summary   = compute_summary(era[0])
merra_corr_summary = compute_summary(merra[0])
clm_corr_summary   = compute_summary(clm[0])
amip_corr_summary  = compute_summary(amip[0])

obs_mi_summary   = compute_summary(obs[1])
era_mi_summary   = compute_summary(era[1])
merra_mi_summary = compute_summary(merra[1])
clm_mi_summary   = compute_summary(clm[1])
amip_mi_summary  = compute_summary(amip[1])

# To display the summaries, loop through the dictionary and print each one.
# for pair_name, summary in obs_corr_summary.items():
#     print(f"Summary for {pair_name}:")
#     print(summary)
#     print("\n")

In [ ]:
obs_nmi_summary   = compute_summary(obs[0])
era_nmi_summary   = compute_summary(era[0])
merra_nmi_summary = compute_summary(merra[0])
clm_nmi_summary   = compute_summary(clm[0])
amip_nmi_summary  = compute_summary(amip[0])
obs_nmi_summary

In [ ]:
obs_nmi_nl_summary   = compute_summary(obs[1])
era_nmi_nl_summary   = compute_summary(era[1])
merra_nmi_nl_summary = compute_summary(merra[1])
clm_nmi_nl_summary   = compute_summary(clm[1])
amip_nmi_nl_summary  = compute_summary(amip[1])
obs_nmi_nl_summary

In [ ]:
obs_nmi_l_summary   = compute_summary(obs[2])
era_nmi_l_summary   = compute_summary(era[2])
merra_nmi_l_summary = compute_summary(merra[2])
clm_nmi_l_summary   = compute_summary(clm[2])
amip_nmi_l_summary  = compute_summary(amip[2])
obs_nmi_l_summary

In [ ]:
obs_te_x_y_summary   = compute_summary(obs[3])
era_te_x_y_summary   = compute_summary(era[3])
merra_te_x_y_summary = compute_summary(merra[3])
clm_te_x_y_summary   = compute_summary(clm[3])
amip_te_x_y_summary  = compute_summary(amip[3])

In [ ]:
obs_te_y_x_summary   = compute_summary(obs[4])
era_te_y_x_summary   = compute_summary(era[4])
merra_te_y_x_summary = compute_summary(merra[4])
clm_te_y_x_summary   = compute_summary(clm[4])
amip_te_y_x_summary  = compute_summary(amip[4])

In [27]:
dirout='/projects/COLA/land/skim/AMERIbase/out_v2/corr_output/'
# datasets = [obs[0],era[0],merra[0]]
datasets = [clm[0],amip[0]]
# names    = ['obs','era','merra']
names    = ['clm','amip']
for i in range(0,2):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)


In [ ]:
cd /projects/COLA/land/skim/FLUNXET/AMERIbase/out_v2

In [ ]:
dirout='/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/nmi_output_v2/'
datasets = [obs[1],era[1],merra[1],clm[1],amip[1]]
names    = ['obs','era','merra','clm','amip']
for i in range(0,5):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)


In [31]:
cd /projects/COLA/land/skim/AMERIbase/out_v2

/projects/COLA/land/skim/AMERIbase/out_v2


In [37]:
ls -rlt

total 625
-rw-rw----+ 1 skim307 proj-cola 12694 Aug  8 05:05 obs_swc_rad.csv
-rw-rw----+ 1 skim307 proj-cola 12713 Aug  8 05:05 obs_swc_le.csv
-rw-rw----+ 1 skim307 proj-cola 12695 Aug  8 05:05 obs_swc_sh.csv
-rw-rw----+ 1 skim307 proj-cola 12716 Aug  8 05:05 obs_swc_gh.csv
-rw-rw----+ 1 skim307 proj-cola 12681 Aug  8 05:05 obs_rad_le.csv
-rw-rw----+ 1 skim307 proj-cola 12694 Aug  8 05:05 obs_rad_sh.csv
-rw-rw----+ 1 skim307 proj-cola 12692 Aug  8 05:05 obs_rad_gh.csv
-rw-rw----+ 1 skim307 proj-cola 12698 Aug  8 05:05 obs_le_sh.csv
-rw-rw----+ 1 skim307 proj-cola 12697 Aug  8 05:05 obs_le_gh.csv
-rw-rw----+ 1 skim307 proj-cola 12703 Aug  8 05:05 obs_sh_gh.csv
-rw-rw----+ 1 skim307 proj-cola 12753 Aug  8 05:05 era_swc_rad.csv
-rw-rw----+ 1 skim307 proj-cola 12766 Aug  8 05:05 era_swc_le.csv
-rw-rw----+ 1 skim307 proj-cola 12749 Aug  8 05:05 era_swc_sh.csv
-rw-rw----+ 1 skim307 proj-cola 12743 Aug  8 05:05 era_swc_gh.csv
-rw-rw----+ 1 skim307 proj-cola 12753 Aug  8 05:05 era_rad_le.csv
-

In [36]:
cd nmi_nonlinear_output

/projects/COLA/land/skim/AMERIbase/out_v2/nmi_nonlinear_output


In [33]:
dirout='/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/nmi_output_v2/'
num=0
datasets = [obs[num],era[num],merra[num],clm[num],amip[num]]
names    = ['obs','era','merra','clm','amip']
for i in range(0,5):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)


In [ ]:
dirout='/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/nmi_nonlinear_output_v2/'
num=1
datasets = [obs[num],era[num],merra[num],clm[num],amip[num]]
names    = ['obs','era','merra','clm','amip']
for i in range(0,5):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)


In [ ]:
dirout='/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/nmi_linear_output_v2/'
num=2
datasets = [obs[num],era[num],merra[num],clm[num],amip[num]]
names    = ['obs','era','merra','clm','amip']
for i in range(0,5):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)


In [ ]:
dirout='/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/te_x_y_output_v2/'
num=3
datasets = [obs[num],era[num],merra[num],clm[num],amip[num]]
names    = ['obs','era','merra','clm','amip']
for i in range(0,5):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)


In [ ]:
dirout='/projects/COLA/land/skim/FLUXNET/AMERIbase/out_v2/te_y_x_output_v2/'
num=4
datasets = [obs[num],era[num],merra[num],clm[num],amip[num]]
names    = ['obs','era','merra','clm','amip']
for i in range(0,5):
    for pair_name, df in datasets[i].items():
        df.to_csv(dirout+names[i]+f'_{pair_name}.csv', index=False,float_format='%.4g', na_rep=-9999)
